# Orbit Propagation Test

In [2]:
# Load tudatpy modules
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.numerical_simulation import environment_setup, propagation_setup
from tudatpy.kernel.astro import element_conversion
from tudatpy.kernel import constants
from tudatpy.util import result2array

In [75]:
# Load other libraries
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load spice kernels
spice.load_standard_kernels()

## Environment Setup

In [41]:
# Add Celestial bodies
bodies_to_create = ["Earth", "Moon"]
global_frame_origin = "Moon"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(bodies_to_create, global_frame_origin, global_frame_orientation)

In [42]:
# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [43]:
# Add vehicle object to system of bodies
bodies.create_empty_body( "Vehicle" )
# Change parameters of Vehicle
bodies.get_body( "Vehicle" ).mass = 100.0

## Propagation Setup

In [44]:
# Define propagation
bodies_to_propagate = ["Vehicle"]
central_bodies = ["Moon"]

In [45]:
# Define accelerations acting on Vehicle
acceleration_settings_vehicle = dict(
    Moon=[propagation_setup.acceleration.spherical_harmonic_gravity(4, 0)],
    Earth=[propagation_setup.acceleration.point_mass_gravity()])
acceleration_settings = {"Vehicle": acceleration_settings_vehicle}

In [46]:
# Create acceleration models
acceleration_models = propagation_setup.create_acceleration_models(bodies, acceleration_settings, bodies_to_propagate, central_bodies)

In [62]:
# Set initial conditions for the Vehicle that will be propagated in this simulation.
# The initial conditions are given in Keplerian elements and later on converted to Cartesian elements. Use SI units.
moon_gravitational_parameter = bodies.get("Moon").gravitational_parameter

initial_state = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=moon_gravitational_parameter,
    semi_major_axis=2000e3,
    eccentricity=0,
    inclination=np.deg2rad(0),
    argument_of_periapsis=np.deg2rad(0),
    longitude_of_ascending_node=np.deg2rad(0),
    true_anomaly=np.deg2rad(0),
)

initial_state

array([ 2.00000000e+06,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        1.56569475e+03,  0.00000000e+00])

In [105]:
# Define time settings
simulation_start_epoch = 0.0
simulation_end_epoch = 24*5*3600.0

In [106]:
# Integration settings
integrator_settings = propagation_setup.integrator.runge_kutta_4(2.0) # Step size is 2 seconds
termination_settings = propagation_setup.propagator.time_termination(simulation_end_epoch)
dependent_variables_to_save = [propagation_setup.dependent_variable.total_acceleration("Vehicle")]

propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_state,
    simulation_start_epoch,
    integrator_settings,
    termination_settings,
    output_variables=dependent_variables_to_save
)

## Propagation Dynamics

In [107]:
# Create simulation object and propagate dynamics.
dynamics_simulator = numerical_simulation.create_dynamics_simulator(bodies, propagator_settings)
results = dynamics_simulator.propagation_results

In [108]:
# Value Results
states = result2array(results.state_history)
states

array([[ 0.00000000e+00,  2.00000000e+06,  0.00000000e+00, ...,
        -0.00000000e+00,  1.56569475e+03,  0.00000000e+00],
       [ 2.00000000e+00,  1.99999755e+06,  3.13138824e+03, ...,
        -2.45197180e+00,  1.56569285e+03,  5.43826249e-05],
       [ 4.00000000e+00,  1.99999019e+06,  6.26276884e+03, ...,
        -4.90393740e+00,  1.56568711e+03,  1.09499345e-04],
       ...,
       [ 4.31996000e+05,  1.13095915e+06, -1.64727704e+06, ...,
         1.29034072e+03,  8.87860563e+02, -3.89013990e+01],
       [ 4.31998000e+05,  1.13353845e+06, -1.64549930e+06, ...,
         1.28894980e+03,  8.89883207e+02, -3.88502778e+01],
       [ 4.32000000e+05,  1.13611495e+06, -1.64371751e+06, ...,
         1.28755571e+03,  8.91903670e+02, -3.87990605e+01]])

In [109]:
# Define the radius of the sphere
r = 1737.4e3

# Define the angles for the phi and theta parameters
phi = np.linspace(0, 2 * np.pi, 20)
theta = np.linspace(0, np.pi, 20)

# Create a meshgrid of phi and theta values
phi, theta = np.meshgrid(phi, theta)

# Calculate the x, y, and z coordinates for each point on the sphere
xM = r * np.sin(theta) * np.cos(phi)
yM = r * np.sin(theta) * np.sin(phi)
zM = r * np.cos(theta)

In [110]:
%matplotlib qt

# Define a 3D figure using pyplot
fig = plt.figure(figsize=(6,6), dpi=125)
ax = fig.add_subplot(111, projection='3d')

# Plot the positional state history
ax.plot(states[:, 1], states[:, 2], states[:, 3], label=bodies_to_propagate[0], linestyle='-.')
ax.plot_surface(xM, yM, zM, color='gray', alpha=0.5)
ax.scatter(0.0, 0.0, 0.0, label="Moon", marker='o', color='gray')

# Add the legend and labels, then show the plot
ax.legend()
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
plt.show()